### Basic example on how to use OMERO Python language bindings
See https://docs.openmicroscopy.org/latest/omero/developers/Python.html

### Import Packages required to connect to OMERO

In [ ]:
from omero.gateway import BlitzGateway
from getpass import getpass
try:
    input = raw_input
except NameError:  # Python 3
    pass

### Create a connection to an OMERO server

In [ ]:
HOST = 'workshop.openmicroscopy.org'
PORT = 4064
conn = BlitzGateway(input("Username: "),
                    getpass("OMERO Password: "),
                    host=HOST, port=PORT)
conn.connect()

# Using secure connection.
# By default, once we have logged in, data transfer is not encrypted
# (faster)
# To use a secured connection, call setSecure(True):
conn.setSecure(True)

### Read Data method

In [ ]:
def print_obj(obj, indent=0):
    """
    Helper method to display info about OMERO objects.
    Not all objects will have a "name" or owner field.
    """
    print("""%s%s:%s  Name:"%s" (owner=%s)""" % (
        " " * indent,
        obj.OMERO_CLASS,
        obj.getId(),
        obj.getName(),
        obj.getOwnerOmeName()))

### Get Projects/Datasets/Images of the current User

In [ ]:
my_exp_id = conn.getUser().getId()
default_group_id = conn.getEventContext().groupId
for project in conn.getObjects("Project",
                               opts={'owner': my_exp_id,
                                     'group': default_group_id,
                                     'order_by': 'lower(obj.name)',
                                     'limit': 5, 'offset': 0}):
    print_obj(project)
    # We can get Datasets with listChildren, since we have the Project already.
    # Or conn.getObjects("Dataset", opts={'project', id}) if we have Project ID
    for dataset in project.listChildren():
        print_obj(dataset, 2)
        for image in dataset.listChildren():
            print_obj(image, 4)

### Close the connection to the OMERO server

In [ ]:
conn.close()